입력 설정
- input_dir: CSV 파일이 저장된 디렉토리 경로
- output_csv: 앙상블 결과를 저장할 파일 경로
- iou_threshold: IOU 기준값(박스/마스크 병합 기준)
- vote_threshold: 특정 예측을 유지하기 위한 최소 모델 투표 수

RLE 처리 함수
- decode_rle: RLE 데이터를 binary mask로 변환
- encode_rle: binary mask를 RLE 데이터로 변환

앙상블 수행
- 동일한 image_name과 class에 대한 RLE를 IOU와 다수결 기준으로 병합
- IOU가 기준치를 넘는 마스크만 병합
- 다수결 기준(vote_threshold)을 만족하는 마스크만 최종 결과에 포함

결과 저장
- 결과를 output_csv로 저장

In [1]:
import os
import pandas as pd
import numpy as np
import torch
from glob import glob
from tqdm import tqdm
from collections import defaultdict

In [2]:
# 입력 설정

# CSV 파일이 위치한 디렉토리 경로
input_dir = "/data/ephemeral/home/kjh2/level2-cv-semanticsegmentation-cv-13-lv3/inference/"

# 결과를 저장할 파일명
output_csv = "csv_ensemble_hard_1.csv"  

# 병합할 RLE 간 IOU 임계값
iou_threshold = 0.5  

# 최소 투표 수
vote_threshold = 2

In [3]:
# GPU/CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
def decode_rle(rle, shape):
    #RLE 데이터를 디코딩해 binary mask를 생성    
    if not rle or rle == '' or pd.isna(rle):  # NaN 값 처리
        return torch.zeros(shape, dtype=torch.uint8, device=device)
    
    # 문자열로 변환 후 처리
    s = list(map(int, str(rle).split()))
    starts, lengths = s[0::2], s[1::2]
    starts = torch.tensor(starts, device=device) - 1
    ends = starts + torch.tensor(lengths, device=device)

    mask = torch.zeros(shape[0] * shape[1], dtype=torch.uint8, device=device)
    for start, end in zip(starts, ends):
        mask[start:end] = 1
    return mask.view(shape)

def encode_rle(mask):
    #binary mask를 RLE 형식으로 변환 (벡터화 버전)
    mask = mask.flatten().cpu().numpy()  # 1D 배열로 변환
    pixels = np.concatenate([[0], mask, [0]])  # 경계처리를 위해 앞뒤에 0 추가
    diffs = np.where(pixels[1:] != pixels[:-1])[0]  # 값이 바뀌는 위치 찾기
    starts = diffs[::2] + 1  # 1의 시작 위치 (1-based 인덱스)
    lengths = diffs[1::2] - diffs[::2]  # 1의 길이
    rle = " ".join(f"{start} {length}" for start, length in zip(starts, lengths))
    return rle

def calculate_iou(mask1, mask2):
    #두 binary mask 간의 IOU를 계산
    intersection = (mask1 & mask2).sum().float()
    union = (mask1 | mask2).sum().float()
    if union == 0:
        return torch.tensor(0.0, device=device)
    return intersection / union

def ensemble_csvs(csv_files, iou_threshold=0.5, vote_threshold=2):
    #CSV 파일들을 앙상블하여 최종 결과 생성
    all_predictions = defaultdict(list)

    # 모든 CSV 파일 로드
    for csv_file in tqdm(csv_files, desc="CSV 파일 로딩 중"):
        df = pd.read_csv(csv_file)
        for _, row in df.iterrows():
            image_name = row['image_name']
            class_name = row['class']
            rle = row['rle']
            all_predictions[(image_name, class_name)].append(rle)
    
    # 앙상블 결과 저장
    ensemble_results = []
    for (image_name, class_name), rles in tqdm(all_predictions.items(), desc="앙상블 처리 중"):
        masks = [decode_rle(rle, shape=(2048, 2048)) for rle in rles]  # 모든 RLE를 binary mask로 변환
        
        final_mask = torch.zeros_like(masks[0], dtype=torch.uint8)
        for i in range(len(masks)):
            vote_count = 0
            temp_mask = torch.zeros_like(masks[0], dtype=torch.uint8)
            for j in range(len(masks)):
                iou = calculate_iou(masks[i], masks[j])
                if iou >= iou_threshold:
                    vote_count += 1
                    temp_mask |= masks[j]  # 겹치는 영역을 병합
            if vote_count >= vote_threshold:
                final_mask |= temp_mask  # 최종 마스크에 반영
        
        if final_mask.sum() > 0:
            ensemble_rle = encode_rle(final_mask)
            ensemble_results.append({
                'image_name': image_name,
                'class': class_name,
                'rle': ensemble_rle
            })
    
    return pd.DataFrame(ensemble_results)

In [ ]:
# 실행 부분
if __name__ == "__main__":
    # 입력 디렉토리 내의 모든 CSV 파일 경로를 가져옴
    csv_files = glob(os.path.join(input_dir, '*.csv'))
    if not csv_files:
        raise ValueError(f"입력 디렉토리에서 CSV 파일을 찾을 수 없음: {input_dir}")
    
    print(f"CSV 파일 수: {len(csv_files)}")
    for csv_file in csv_files:
        print(f"- {csv_file}")
    
    # 앙상블 수행
    ensemble_result_df = ensemble_csvs(
        csv_files=csv_files,
        iou_threshold=iou_threshold,
        vote_threshold=vote_threshold
    )

    # 결과 저장
    ensemble_result_df.to_csv(output_csv, index=False)
    print(f"앙상블 끝!!!!!!: {output_csv}")

발견된 CSV 파일 수: 4
- /data/ephemeral/home/kjh2/level2-cv-semanticsegmentation-cv-13-lv3/inference/UPerNet_efficientnet-b7_2048_epoch100.csv
- /data/ephemeral/home/kjh2/level2-cv-semanticsegmentation-cv-13-lv3/inference/segformer_from_mmseg.csv
- /data/ephemeral/home/kjh2/level2-cv-semanticsegmentation-cv-13-lv3/inference/UPerNet mit_b5 1024.csv
- /data/ephemeral/home/kjh2/level2-cv-semanticsegmentation-cv-13-lv3/inference/SAM2UNet_1024p_1e-4_amp_best.csv


앙상블 처리 중: 100%|██████████| 8352/8352 [08:47<00:00, 15.83it/s]


앙상블 결과가 저장되었습니다: csv_ensemble_hard_1.csv
